In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import itertools
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
%matplotlib inline

In [2]:
X = np.load('../../X_train.npz')['arr_0']
y = np.load('../../Y_train.npz')['arr_0']
X_test = np.load('../../X_test.npz')['arr_0']

In [3]:
def err1(y_predicted, y_true):
    return 'e1', np.mean(np.abs(y_true.get_label() - y_predicted))

def err2(y_predicted, y_true):
    return 'e2', np.mean((np.abs(y_true.get_label() - y_predicted)) / y_true.get_label())

In [4]:
def mymae(y_true, y_pred):
    grad = np.tanh(y_pred - y_true)
    hess = 1 - grad * grad
    return grad, hess

In [5]:
params_xgbr = {
    'objective': mymae
}
params = {
    'booster':['dart'],
    'n_jobs':[1],
    'learning_rate':[0.001, 0.05, 0.1],
    'max_depth':[6],
    'subsample':[0.5, 0.75, 1],
    'tree_method':['gpu_exact'],
    'predictor': ['gpu_predictor'], 
    'rate_drop':[0.5, 0.25, 0.1],
    'n_estimators': [1000],
    'n_gpu':[1],
    'objective': [mymae]
}

In [6]:
rd = np.random.permutation(47500)
X_tmp, y_tmp = X[rd[:10000]], y[rd[:10000]][:, 0]
rd = None

In [ ]:
keys, values = zip(*params.items())
e_in = []
e_val = []
i = 0
for v in itertools.product(*values):
    j = 0
    kf = KFold(n_splits=5, shuffle=False)
    for train_idx, val_idx in kf.split(X_tmp):
        param = dict(zip(keys, v))
        print(i, j)
        print(v)
        X_train, X_val = X_tmp[train_idx], X_tmp[val_idx]
        y_train, y_val = y_tmp[train_idx], y_tmp[val_idx]
        
        model = XGBRegressor(**param)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric=err1,
                  early_stopping_rounds=5)
        a, b = err1(model.predict(X_train), y_train)
        e_in.append(b)
        print(f'e_in: {b}')
        a, b = err1(model.predict(X_val), y_val)
        e_val.append(b)
        print(f'e_val: {b}')
        
        del(param, X_train, X_val, y_train, y_val, model, a, b)
        j += 1
    i += 1

In [7]:
import gc
gc.enable()

In [8]:
model = XGBRegressor(predictor='gpu_predictor', tree_method='gpu_exact', gpu_id=2)
model.fit(X_tmp[:7500], y_tmp[:7500])
gc.collect()

/home/student/07/b07902111/python-packages/xgboost/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[02:21:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBoostError: [02:21:33] /workspace/src/tree/updater_gpu.cu:605: grow_gpu exception: [02:21:33] /workspace/src/common/common.h:41: /workspace/src/tree/../common/device_helpers.cuh: 339: out of memory
Stack trace:
  [bt] (0) /home/student/07/b07902111/python-packages/xgboost/lib/python3.7/site-packages/xgboost/./lib/libxgboost.so(dh::ThrowOnCudaError(cudaError, char const*, int)+0x3b5) [0x7f87c1e5fb95]
  [bt] (1) /home/student/07/b07902111/python-packages/xgboost/lib/python3.7/site-packages/xgboost/./lib/libxgboost.so(xgboost::tree::GPUMaker::SetupOneTimeData(xgboost::DMatrix*)+0x2f7) [0x7f87c1ef3857]
  [bt] (2) /home/student/07/b07902111/python-packages/xgboost/lib/python3.7/site-packages/xgboost/./lib/libxgboost.so(xgboost::tree::GPUMaker::Update(xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*, xgboost::DMatrix*, std::vector<xgboost::RegTree*, std::allocator<xgboost::RegTree*> > const&)+0x35ff) [0x7f87c1ef7acf]
  [bt] (3) /home/student/07/b07902111/python-pac

In [9]:
import gc

In [12]:
gc.collect()

20

In [13]:
gc.callbacks

[]